In [1]:
import os
import numpy as np
from keras import models, layers, optimizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
keras.__version__

d:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.2.4'

In [2]:
imdb_dir = 'F:\\study\\ml\\DataSet\\aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

In [3]:
labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf-8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [5]:
texts[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [17]:
maxlen = 100
train_samples = 200
validation_smaples = 10000
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('found %s word vector' % len(word_index))

found 88582 word vector


In [18]:
data = pad_sequences(sequences, maxlen=maxlen)

In [19]:
data.shape

(25000, 100)

In [23]:
labels = np.asarray(labels)
print('shape of data tensor', data.shape)
print('shaoe of label tensor', labels.shape)

shape of data tensor (25000, 100)
shaoe of label tensor (25000,)


In [24]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [47]:
x_train = data[:train_samples]
y_train = labels[:train_samples]

x_val = data[train_samples:train_samples + validation_smaples]
y_val = labels[train_samples:train_samples + validation_smaples]

In [29]:
glove_dir = 'F:\\study\\ml\\DataSet\\glove.6B'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('found %s word vector ' % len(embeddings_index))

found 400000 word vector 


In [30]:
len(word_index)

88582

In [31]:
embeddings_dim = 100

embeddings_matrix = np.zeros((max_words, embeddings_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

In [49]:
embeddings_matrix.shape

(10000, 100)

In [51]:
embeddings_matrix[1]

array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
        0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
        0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
       -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
        0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
       -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
       -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
        0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
       -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
        0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
       -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
       -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
       -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
       -0.19358   , -0.074575  ,  0.23353   , -0.052062  , -0.22

In [39]:
from keras.layers import Embedding, Flatten, Dense

In [40]:
model = models.Sequential()
model.add(Embedding(max_words, embeddings_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                320032    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.layers[0].set_weights([embeddings_matrix])
model.layers[0].trainable = False

In [53]:
x_train[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  134,   11,   17,  215,    3,  353,
          31,   98, 3224,    9,    6,   52,  438,   14,   10,  374,    9,
          10,  216,    9,   41, 1114,  150,  594,   20, 4273,    2,  444,
           1,   17,    9,   13,  395,   31,    1,  169,  230,   12, 1038,
           2,  523, 1604,    2,  148,    9,  215,   14,  160,   14,   12,
          17,    9,  124,  120,    1,  983,   12, 1228, 2235, 3827,   16,
        1604],
       [ 170,    4,   11, 6454,   14,  227,   14,    1,  111,   70,    9,
           6,  217,  695, 2965,    2,   35,  858,  221,    2,  395,  419,
          10,  121,   42,  157, 7292,   10,   52,   73,  822,  256,   77,
         374,  229,   41,   11,  589,  841,   22,   78,   89,  434,  126,
          55,   18,   4

In [48]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))

Train on 200 samples, validate on 10000 samples
Epoch 1/10
200/200 [==============================] - 1s 3ms/step - loss: 1.6063 - acc: 0.5200 - val_loss: 0.6847 - val_acc: 0.5571
Epoch 2/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5007 - acc: 0.7850 - val_loss: 0.8800 - val_acc: 0.4978
Epoch 3/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3377 - acc: 0.8650 - val_loss: 0.9751 - val_acc: 0.4977
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.3678 - acc: 0.8150 - val_loss: 1.1090 - val_acc: 0.4957
Epoch 5/10
200/200 [==============================] - 1s 3ms/step - loss: 0.2496 - acc: 0.9200 - val_loss: 0.7184 - val_acc: 0.5780
Epoch 6/10
200/200 [==============================] - 1s 3ms/step - loss: 0.1338 - acc: 0.9950 - val_loss: 0.7680 - val_acc: 0.5557
Epoch 7/10
200/200 [==============================] - 1s 3ms/step - loss: 0.0823 - acc: 0.9950 - val_loss: 0.8170 - val_acc: 0.5612
Epoch 8/10
200/200 [========

In [56]:
test_dir = os.path.join(imdb_dir, 'test')
test_labels = []
test_texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf-8')
            test_texts.append(f.read())
            f.close()
            if label_type == 'neg':
                test_labels.append(0)
            else:
                test_labels.append(1)

In [58]:
test_sequences = tokenizer.texts_to_sequences(test_texts)
x_test = pad_sequences(test_sequences, maxlen=maxlen)
y_test = np.asarray(test_labels)

In [59]:
model.evaluate(x_test, y_test)

25000/25000 [==============================] - 1s 50us/step


[0.7921250235080719, 0.57536]